In [1]:
print("hello")

hello


In [2]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM

# ==========================================
# 1. 設定 & 定数
# ==========================================
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
SAVE_PATH = "transformer_creativity_probe_enhanced.pt"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TARGET_LAYER = 24  # 特徴量を抽出する層 (Qwen-7Bの中深層)
Input_Dim = 3584   # Qwen-2.5-7B の隠れ層次元数

# 再現性確保
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

In [3]:
# ==========================================
# 2. モデル定義 (Probe)
# ==========================================
class TransformerProbe(nn.Module):
    def __init__(self, input_dim, d_model=256, nhead=4):
        super().__init__()
        self.project = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=512, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)
        self.head = nn.Linear(d_model, 1)
        
    def forward(self, x):
        # x: [Batch, Seq, Input_Dim]
        x = self.project(x)
        x = self.transformer(x)
        x = x.mean(dim=1) # Global Average Pooling
        return self.head(x)

In [4]:
# ==========================================
# 3. データセット生成 (Enhanced)
# ==========================================
def generate_enhanced_dataset(model, tokenizer, n_samples=3000):
    model.eval()
    print(f"Generating {n_samples} samples with Research-Aware strategies...")

    # --- A. Positive Prompts (Label = 1.0) ---
    # 斬新な発想、異分野融合、詩的な表現
    prompts_pos = [
        "Write a short poem using only simple words:",
        "Describe a complex emotion using a metaphor:",
        "Compose a mysterious story opening:",
        # ★ 研究: 異分野融合・新規性
        "Propose a groundbreaking research idea combining quantum physics and LLMs:",
        "Suggest a novel method to train AI without using any human data:",
        "Invent a new neural network architecture inspired by biological ecosystems:",
        "Write an abstract for a paper that challenges the scaling laws of transformers:",
    ]

    # --- B. Negative Prompts (Label = 0.0) ---
    # 退屈、定型文、ありきたりな研究、専門用語サラダ
    prompts_boring = [
        "State a dry fact about history:",
        "Explain a basic grammatical rule:",
        "Describe the process of boiling water:",
    ]
    prompts_instruction = [
        "Give a polite response declining a request:",
        "Say 'Here is the summary' in a formal way:",
        "Write a standard AI assistant response:",
    ]
    # ★ 研究: ありきたり (Cliché)
    prompts_cliche_research = [
        "Describe a standard MNIST digit classification experiment:",
        "Explain how to fine-tune BERT for sentiment analysis:",
        "Write a generic introduction about the importance of big data:",
        "State the definition of supervised learning from a textbook:",
    ]
    
    # ★ 研究: 専門用語サラダ (Fake Jargon) 用の単語リスト
    jargon_prefixes = ["Hyper-", "Quantum", "Neuro-", "Multi-", "Meta-", "Cyber-", "Deep"]
    jargon_nouns = ["Entropy", "Optimization", "Tensors", "Gradients", "Blockchain", "Synergy", "Paradigm"]
    jargon_verbs = ["disrupts", "leverages", "synthesizes", "encrypts", "validates"]

    data = []
    labels = []

    # 生成ループ
    # Positive (40%)
    for _ in tqdm(range(int(n_samples * 0.4)), desc="Positive"):
        prompt = random.choice(prompts_pos)
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=64, temperature=0.95, do_sample=True, pad_token_id=tokenizer.eos_token_id)
        text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()
        data.append(text)
        labels.append(1.0)

    # Negative (60%)
    for _ in tqdm(range(int(n_samples * 0.6)), desc="Negative"):
        r = random.random()
        
        if r < 0.25: # Boring
            prompt = random.choice(prompts_boring)
            inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                out = model.generate(**inputs, max_new_tokens=64, temperature=0.1, do_sample=True, pad_token_id=tokenizer.eos_token_id)
            text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()
            
        elif r < 0.50: # Instruction
            prompt = random.choice(prompts_instruction)
            inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                out = model.generate(**inputs, max_new_tokens=64, temperature=0.1, do_sample=True, pad_token_id=tokenizer.eos_token_id)
            text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()

        elif r < 0.75: # Cliché Research (ありきたりな研究)
            prompt = random.choice(prompts_cliche_research)
            inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
            with torch.no_grad():
                out = model.generate(**inputs, max_new_tokens=64, temperature=0.1, do_sample=True, pad_token_id=tokenizer.eos_token_id)
            text = tokenizer.decode(out[0], skip_special_tokens=True).replace(prompt, "").strip()
            
        else: # Fake Jargon (専門用語サラダ) - 生成ではなく合成で作る
            p1 = random.choice(jargon_prefixes) + random.choice(jargon_nouns)
            p2 = random.choice(jargon_prefixes) + random.choice(jargon_nouns)
            v = random.choice(jargon_verbs)
            text = f"The {p1} {v} the {p2} for optimal performance within the stochastic framework."
            
        data.append(text)
        labels.append(0.0)

    return data, labels

In [5]:
# ==========================================
# 4. 特徴量抽出ヘルパー
# ==========================================
def extract_features(model, tokenizer, texts, batch_size=16):
    model.eval()
    features = []
    
    print("Extracting Hidden States...")
    # バッチ処理で高速化
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i : i+batch_size]
        # 空文字対策
        batch_texts = [t if t.strip() else "empty" for t in batch_texts]
        
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(DEVICE)
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        
        # 指定層の特徴量を取得 [Batch, Seq, Dim]
        h = outputs.hidden_states[TARGET_LAYER].float().cpu()
        
        for j in range(len(batch_texts)):
            # パディング部分(mask=0)を除外するのが理想だが、今回は簡略化のためそのまま使用
            # (ProbeがTransformerなのでAttentionで無視することを期待、あるいはTruncationで対処)
            features.append(h[j]) 
            
    return features

In [6]:
# ==========================================
# 5. 厳密評価ベンチマーク
# ==========================================
def strict_benchmark(probe, base_model, tokenizer):
    print("\n=== 🛑 Running Strict Research Benchmark ===")
    
    cases = [
        # --- Research Domain (最重要) ---
        ("Research", "Cliché", "Using deep learning to classify images of cats and dogs using CNNs."),
        ("Research", "Novel", "Training LLMs using simulated debates between historical figures to improve reasoning."),
        ("Research", "Nonsense", "Fine-tuning a neural network on the taste of the color blue using quantum gradients."),
        
        # --- Story Domain (確認用) ---
        ("Story", "Cliché", "A detective solves a murder case in a small town."),
        ("Story", "Novel", "A murder mystery where the detective is the ghost of the victim."),
    ]
    
    results = []
    probe.eval()
    
    print(f"{'Domain':10s} | {'Type':10s} | {'Logit':8s} | Text")
    print("-" * 60)
    
    for domain, type_, text in cases:
        inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            out = base_model(**inputs, output_hidden_states=True)
            h = out.hidden_states[TARGET_LAYER].squeeze(0).float() # [Seq, Dim]
            
            # Probe
            logit = probe(h.unsqueeze(0).to(DEVICE)).item()
            
        results.append({"Domain": domain, "Type": type_, "Score": logit})
        print(f"{domain:10s} | {type_:10s} | {logit:8.4f} | {text[:40]}...")
        
    return pd.DataFrame(results)

In [7]:
# ==========================================
# 6. メイン実行
# ==========================================
# --- A. ベースモデル準備 ---
print(f"Loading Base Model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

# 特徴抽出だけなので軽量ロード推奨 (float16)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, device_map="auto"
)

# --- B. データ作成 & 特徴抽出 ---
texts, labels = generate_enhanced_dataset(base_model, tokenizer, n_samples=3000)

features = extract_features(base_model, tokenizer, texts)
    
# パディングしてテンソル化 (Batch化のため長さを揃える必要があるが、
# ここでは可変長のままDatasetに入れて、Collatefnでパディングするか、
# 簡易的に max_len で切ったものを使う。
# extract_featuresですでにtensor化されているので、pad_sequenceを使う)
from torch.nn.utils.rnn import pad_sequence
X_data = pad_sequence(features, batch_first=True).to(DEVICE) # [N, MaxSeq, Dim]
y_data = torch.tensor(labels).float().unsqueeze(1).to(DEVICE)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.15)

dataset = torch.utils.data.TensorDataset(X_train, y_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# --- C. 学習 (Training) ---
print("\nStarting Probe Training...")
probe = TransformerProbe(Input_Dim).to(DEVICE)
optimizer = optim.Adam(probe.parameters(), lr=5e-5)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(15):
    probe.train()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        # Maskingは省略(Probeが学習で無視することを期待)
        outputs = probe(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Loss = {total_loss/len(dataloader):.4f}")

# --- D. 厳密評価 (Strict Evaluation) ---
df_res = strict_benchmark(probe, base_model, tokenizer)

# 判定ロジック
research_res = df_res[df_res["Domain"] == "Research"]
novel_score = research_res[research_res["Type"] == "Novel"]["Score"].values[0]
cliche_score = research_res[research_res["Type"] == "Cliché"]["Score"].values[0]
nonsense_score = research_res[research_res["Type"] == "Nonsense"]["Score"].values[0]

print("\n=== 📝 Final Verdict ===")
print(f"Research Novel:    {novel_score:.2f}")
print(f"Research Cliché:   {cliche_score:.2f}")
print(f"Research Nonsense: {nonsense_score:.2f}")

success = False
if novel_score > cliche_score and novel_score > nonsense_score:
    print("✅ PASS: Novel ideas are rated highest!")
    success = True
elif novel_score > cliche_score:
    print("⚠️ CONDITIONAL PASS: Novel > Cliché, but Nonsense is high.")
    print("   (Acceptable if PPO uses hybrid gating)")
    success = True
else:
    print("❌ FAIL: Probe prefers Cliché or Nonsense.")

# --- E. 保存 (Save) ---
if success:
    checkpoint = {
        "model_state_dict": probe.state_dict(),
        "config": {
            "input_dim": Input_Dim,
            "d_model": 256,
            "nhead": 4,
            "layer_idx": TARGET_LAYER,
            "description": "Enhanced Research-Aware Probe"
        }
    }
    torch.save(checkpoint, SAVE_PATH)
    print(f"\n💾 Model saved to: {SAVE_PATH}")
else:
    print("\n🚫 Model NOT saved due to benchmark failure.")

Loading Base Model: Qwen/Qwen2.5-7B-Instruct


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Generating 3000 samples with Research-Aware strategies...


Negative: 100%|██████████| 1800/1800 [28:21<00:00,  1.06it/s]


Extracting Hidden States...


100%|██████████| 188/188 [00:32<00:00,  5.87it/s]



Starting Probe Training...
Epoch 0: Loss = 0.1410
Epoch 5: Loss = 0.0009
Epoch 10: Loss = 0.0005

=== 🛑 Running Strict Research Benchmark ===
Domain     | Type       | Logit    | Text
------------------------------------------------------------
Research   | Cliché     |   1.4851 | Using deep learning to classify images o...
Research   | Novel      |   8.2337 | Training LLMs using simulated debates be...
Research   | Nonsense   |   4.7422 | Fine-tuning a neural network on the tast...
Story      | Cliché     |   3.5965 | A detective solves a murder case in a sm...
Story      | Novel      |   8.3346 | A murder mystery where the detective is ...

=== 📝 Final Verdict ===
Research Novel:    8.23
Research Cliché:   1.49
Research Nonsense: 4.74
✅ PASS: Novel ideas are rated highest!

💾 Model saved to: transformer_creativity_probe_enhanced.pt
